# Matlab to .pkl Conversion

The purpose of this notebook is to streamline the conversion of the WIM data from matlab into a useable form. This involves converting first the Matlab table to a struct and then in python converting this struct as well as combining different structs representing different years. Currently years 2011-2019 are combined. The date/times are also converted to a single datetime.


In [4]:
import scipy.io
import pandas as pd
import numpy as np

In [2]:
def load_table_from_struct(table_structure) -> pd.DataFrame():

    # get prepared data structure
    data = table_structure[0, 0]['table']['data']
    # get prepared column names
    data_cols = [name[0] for name in table_structure[0, 0]['columns'][0]]

    # create dict out of original table
    table_dict = {}
    for colidx in range(len(data_cols)):
        if data_cols[colidx] != 'HH':
            table_dict[data_cols[colidx]] = [val[0] for val in data[0, 0][0, colidx]]
    
    return pd.DataFrame(table_dict)

In [3]:
def df_cleaning(df, counting_only):
    
    #If this will be used for counting only, eliminate the axle weights, etc
    if ~counting_only:
        df = df[['FS', 'GW_TOT', 'CLASS', 'HHMMSS', 'JJJJMMTT', 'ZST', 'LENTH', 'CS']]
    
    df['HHMMSS'] = df['HHMMSS'].astype(str) 
    df['HH'] = df['HHMMSS'].str[:-4]
    df['MMSS'] = df['HHMMSS'].str[-4:]
    df['MM'] = df['MMSS'].str[:-2]
    df = df.replace(r'', np.nan, regex=True)
    df = df.fillna(0)
    df['HH'] = df['HH'].astype(int)
    df['MM'] = df['MM'].astype(int)
    df['Date'] = df['JJJJMMTT'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
    df = df.drop(columns=['JJJJMMTT'])
    df['Date'] += pd.to_timedelta(df.HH, unit='h')
    df['Date'] += pd.to_timedelta(df.MM, unit='m')
    df['SS'] = df['MMSS'].str[-2:]
    df['SS'] = df['SS'].astype(int)
    df['Date'] += pd.to_timedelta(df.SS, unit='s')
    df = df.drop(columns=['HH', 'MM', 'SS', 'MMSS', 'HHMMSS'])
    return df

In [7]:
def to_pickle(station, start_year, end_year, all_columns):
    dfs = []
    for year in range(start_year, end_year+1):
        struct = scipy.io.loadmat('{}Data/{}_{}_struct.mat'.format(station, station, year))
        df = load_table_from_struct(struct['{}_struct'.format(station)])
        df = df_cleaning(df, all_columns)
        dfs.append(df)

    df = pd.concat(dfs, ignore_index=True)
    if all_columns:
        df.to_pickle('{}Data/{}_{}_datetime_all.pkl'.format(station, start_year, end_year))
    else:
        df.to_pickle('{}Data/{}_{}_datetime.pkl'.format(station, start_year, end_year))